# 2D Gaussian Model Training - Learning Posterior of φ = θ[1]

This notebook trains a Neural Ratio Estimator (NRE) to learn the **1D posterior distribution p(φ|x)** where φ = θ[1] is the second component of a 2D parameter vector.

## Model
- **Full parameter space**: θ = [θ₀, θ₁] ∈ ℝ² (2D)
- **Parameter of interest**: φ = θ[1] ∈ ℝ (1D)
- **Observations**: x ∈ ℝ^(n_obs × 2) (2D data)
- **Goal**: Learn p(φ|x) = p(θ[1]|x) - a 1D posterior distribution

## Model Setup
- **Model**: `GaussGaussMultiDimModel` with dimension 2
- **Configuration**: `gauss_gauss_2d_marginal1.yml` with `marginal_of_interest=1`
- **Transform**: φ = transform_phi(θ) = θ[1]

## Training Pipeline
1. Load 2D model configuration with `marginal_of_interest=1`
2. Generate synthetic 2D observed data x
3. Learn summary statistics for 2D data → s(x)
4. Train NRE to estimate r(φ, s(x)) = p(s(x)|φ)/p(s(x))
5. Use r(φ, s(x)) to obtain p(φ|x) ∝ p(φ) × r(φ, s(x))

The result is a **1D posterior distribution** for the parameter of interest φ = θ[1], even though the underlying model is 2D.

In [1]:
# =============================================================================
# Main script to run a full end-to-end example for the abcnre package.
# Training script for 2D Gaussian model with focus on second marginal component
# =============================================================================

import jax
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# --- Imports from abcnre package ---
from abcnre.simulation import ABCSimulator
from abcnre.simulation.models import GaussGaussMultiDimModel

# Direct import from registry to avoid potential circular imports
from abcnre.simulation.models.registry import (
    create_model_from_yaml,
    save_model_to_yaml,
    get_example_configs
)

from abcnre.inference.persistence import save_classifier
from abcnre.inference.config import NREConfig, get_nre_config
from abcnre.inference.estimator import NeuralRatioEstimator
from abcnre.inference.networks.base import create_network_from_config
from abcnre.simulation.models.registry import _create_model_from_config
import yaml
%load_ext autoreload
%autoreload 2

In [2]:
output_dir = Path("../results")
simulator_path = output_dir / "gauss_2D_marginal1_simulator.yml"
network_config_path = output_dir / "config_mlp_reduce_on_plateau.yml"

In [3]:
print("--- Step 0: Create Simulator with Observed Data and Learn Summary Statistics ---")
print("🎯 Goal: Learn 1D posterior p(φ|x) where φ = θ[1] from 2D model")

# Load model from YAML configuration for 2D marginal1
config_name = 'gauss_gauss_2d_marginal1'
print(f"\nLoading model from: {config_name}")

# Chemin correct pour le fichier de configuration 2D marginal1
config_path = Path("../../configs/models") / f"{config_name}.yml"
if config_path.exists():
    with open(config_path, "r") as f:
        config_model = yaml.safe_load(f)
else:
    raise FileNotFoundError(f"Configuration file {config_path} does not exist.")

gauss_model = _create_model_from_config(config=config_model, source_path=config_path)
print(f"Model loaded: {gauss_model}")
print(f"Full parameter space dimension: {gauss_model.dim}D (θ ∈ ℝ²)")
print(f"Parameter of interest: φ = θ[{gauss_model.marginal_of_interest}] ∈ ℝ (1D)")

# Generate observed data FIRST - create 2D true parameter
key = jax.random.PRNGKey(42)
key, subkey = jax.random.split(key)
true_theta = np.array([1.5, -0.8])  # 2D parameter vector
phi_true = gauss_model.transform_phi(true_theta)[0]  # Extract scalar from 1D array
print(f"\nTrue parameter vector θ: {true_theta}")
print(f"True parameter of interest φ = θ[{gauss_model.marginal_of_interest}]: {phi_true}")

epsilon_quantile = 1.
observed_data = gauss_model.simulate(subkey, theta=true_theta)
print(f"Observed data shape: {observed_data.shape}")  # Should be (n_obs, 2)
print(f"📊 We will learn p(φ={phi_true:.3f}|x) from these 2D observations")

# Re-import ABCSimulator with updated code
from abcnre.simulation.simulator import ABCSimulator

# Create NEW simulator WITH observed data (important for consistent dimensions)
simulator = ABCSimulator(
    model=gauss_model,
    observed_data=observed_data,
    quantile_distance=epsilon_quantile
)
print(f"Simulator created with epsilon = {simulator.epsilon:.4f}")


# Import config_learner from external YAML file - corrected path
config_learner_path = Path("../../../src/abcnre/preprocessing/configs/summaryDeepSet.yaml")
with open(config_learner_path, "r") as f:
    config_learner = yaml.safe_load(f)

# Learn summary statistics using the simulator
key, key_learn = jax.random.split(key)
simulator.learn_summary_stats(key=key_learn, config_learner=config_learner)
print("✅ Summary statistics learned with DeepSet for i.i.d. data")

# Set the custom summary statistics function directly
# simulator.summary_stats_fn = simple_summary_stats_2d

# Test the summary statistics on observed data
# observed_summary = simulator.summary_stats_fn(observed_data)
# print(f"Observed data shape: {observed_data.shape}")
# print(f"Summary statistics shape: {observed_summary.shape}")
# print(f"Summary statistics: {observed_summary}")
# print(f"  - Sample means: [{observed_summary[0]:.3f}, {observed_summary[1]:.3f}]")
# print(f"  - Sample variances: [{observed_summary[2]:.3f}, {observed_summary[3]:.3f}]")

# # Store the observed summary statistics
# simulator.observed_summary_stats = observed_summary

# # Test on a batch of data
# key_test, key_batch = jax.random.split(key)
# test_thetas, test_xs = gauss_model.sample_theta_x_multiple(key_batch, n_samples=5)
# test_summaries = simulator.summary_stats_fn(test_xs)
# print(f"\nTest batch - data shape: {test_xs.shape}")
# print(f"Test batch - summaries shape: {test_summaries.shape}")

# print("\n✅ Simple analytical summary statistics configured for 2D Gaussian data")
# print("🎯 Summary: [mean_x1, mean_x2, var_x1, var_x2] - 4D summary statistics")
# print("📈 Ready for NRE training: r(φ, s(x)) where s(x) ∈ ℝ⁴")

# simulator

--- Step 0: Create Simulator with Observed Data and Learn Summary Statistics ---
🎯 Goal: Learn 1D posterior p(φ|x) where φ = θ[1] from 2D model

Loading model from: gauss_gauss_2d_marginal1
Model loaded: GaussGaussMultiDimModel(dim=2, n_obs=10, marginal_of_interest=1)
Full parameter space dimension: 2D (θ ∈ ℝ²)
Parameter of interest: φ = θ[1] ∈ ℝ (1D)

True parameter vector θ: [ 1.5 -0.8]
True parameter of interest φ = θ[1]: -0.800000011920929
Observed data shape: (10, 2)
📊 We will learn p(φ=-0.800|x) from these 2D observations
Computing epsilon for 100.0% quantile...
Setting epsilon to infinity (maximum distance).
Simulator created with epsilon = inf
Learning summary statistics with generator pattern...
Loaded config from: preprocessing/configs/summaryDeepSet.yaml
Using LR schedule: constant
Learning rate: 0.0003
Phi dims: [32, 16]
Rho dims: [32, 16]
Training summary statistics with direct objective: min ||φ - s(x)||²
   - Architecture: DeepSet
   - Parameter dimension: 2
   - Summary

In [4]:
key, key_data = jax.random.split(key)
phis, xs = simulator.model.sample_phi_x_multiple(
    key=key_data,
    n_samples=1000)

sumarries = simulator.summary_stats_fn(xs)

corr = np.corrcoef(sumarries.flatten(), phis.flatten())
print(f"Correlation between summary statistics and φ: {corr[0, 1]:.4f}")
print("📊 Correlation indicates how well summary statistics capture φ")

SHAPE X_FORMATTED: (1000, 10, 2)
None
Correlation between summary statistics and φ: 0.9962
📊 Correlation indicates how well summary statistics capture φ
Correlation between summary statistics and φ: 0.9962
📊 Correlation indicates how well summary statistics capture φ


In [5]:
print('--- Step 0.5: Verification of Summary Statistics for 1D Posterior Learning ---')
print("🔍 Checking that s(x) captures information about φ = θ[1]")

key, key_sample = jax.random.split(key)
n_samples = 10000

thetas, xs = simulator.model.sample_theta_x_multiple(key_sample, n_samples=n_samples)
statistics = simulator.summary_stats_fn(xs)

print(f"Theta samples shape: {thetas.shape}")  # (n_samples, 2)
print(f"Data samples shape: {xs.shape}")      # (n_samples, n_obs, 2)
print(f"Summary statistics shape: {statistics.shape}")

# Extract phi values (our 1D parameter of interest)
phi_samples = gauss_model.transform_phi(thetas.T).T.flatten()  # Extract φ = θ[1]
print(f"Phi (parameter of interest) shape: {phi_samples.shape}")  # (n_samples,)

# Check how well summary statistics correlate with phi
if statistics.ndim == 1:
    corr_phi = np.corrcoef(statistics, phi_samples)[0, 1]
    print(f"📈 Correlation between s(x) and φ = θ[{gauss_model.marginal_of_interest}]: {corr_phi:.4f}")
elif statistics.ndim == 2:
    # Multi-dimensional summary statistics
    print("📊 Multi-dimensional summary statistics:")
    for i in range(statistics.shape[1]):
        corr_phi_i = np.corrcoef(statistics[:, i], phi_samples)[0, 1]
        print(f"   s(x)[{i}] ↔ φ correlation: {corr_phi_i:.4f}")
    
    # Overall correlation with best component
    best_corr = max([abs(np.corrcoef(statistics[:, i], phi_samples)[0, 1]) 
                     for i in range(statistics.shape[1])])
    print(f"📈 Best correlation with φ = θ[{gauss_model.marginal_of_interest}]: {best_corr:.4f}")

print(f"✅ Summary statistics contain information needed for learning p(φ|x)")
print(f"🎯 Ready to train NRE for 1D posterior p(φ = θ[{gauss_model.marginal_of_interest}]|x)")

--- Step 0.5: Verification of Summary Statistics for 1D Posterior Learning ---
🔍 Checking that s(x) captures information about φ = θ[1]
SHAPE X_FORMATTED: (10000, 10, 2)
None
SHAPE X_FORMATTED: (10000, 10, 2)
None
Theta samples shape: (10000, 2)
Data samples shape: (10000, 10, 2)
Summary statistics shape: (10000, 1)
Phi (parameter of interest) shape: (10000,)
📊 Multi-dimensional summary statistics:
   s(x)[0] ↔ φ correlation: 0.9965
📈 Best correlation with φ = θ[1]: 0.9965
✅ Summary statistics contain information needed for learning p(φ|x)
🎯 Ready to train NRE for 1D posterior p(φ = θ[1]|x)
Theta samples shape: (10000, 2)
Data samples shape: (10000, 10, 2)
Summary statistics shape: (10000, 1)
Phi (parameter of interest) shape: (10000,)
📊 Multi-dimensional summary statistics:
   s(x)[0] ↔ φ correlation: 0.9965
📈 Best correlation with φ = θ[1]: 0.9965
✅ Summary statistics contain information needed for learning p(φ|x)
🎯 Ready to train NRE for 1D posterior p(φ = θ[1]|x)


In [6]:
# --- Step 1: Display Simulator Configuration for 1D Posterior Learning ---
print("--- Step 1: Configuration for Learning p(φ|x) ---")

print(f"🎯 OBJECTIVE: Learn 1D posterior p(φ|x) where φ = θ[{gauss_model.marginal_of_interest}]")
print(f"📊 Input: 2D observations x with shape {simulator.observed_data.shape}")
print(f"📈 Output: 1D posterior distribution over φ ∈ ℝ")

print(f"\nSimulator epsilon = {simulator.epsilon:.4f}")
print(f"Summary statistics enabled: {simulator.config.get('summary_stats_enabled', False)}")

# Display model properties from metadata
model_args = gauss_model.get_model_args()
print("\nModel configuration:")
for key_arg, value in model_args.items():
    print(f"  {key_arg}: {value}")



--- Step 1: Configuration for Learning p(φ|x) ---
🎯 OBJECTIVE: Learn 1D posterior p(φ|x) where φ = θ[1]
📊 Input: 2D observations x with shape (10, 2)
📈 Output: 1D posterior distribution over φ ∈ ℝ

Simulator epsilon = inf
Summary statistics enabled: True

Model configuration:
  model_type: GaussGaussMultiDimModel
  model_class: GaussGaussMultiDimModel
  model_args: {'mu0': [0.0, 0.0], 'sigma0': [[4.0, 0.0], [0.0, 4.0]], 'sigma': [[0.25, 0.0], [0.0, 0.25]], 'dim': 2, 'n_obs': 10, 'marginal_of_interest': 1}
  metadata: {'parameter_space_dim': 2, 'analytical_posterior': True, 'module': 'abcnre.simulation.models.gauss_gauss'}


In [7]:
# --- Step 1.5: Save Model Configuration to YAML ---
print("\n--- Step 1.5: Save Model Configuration to YAML ---")

# Save the model configuration for reproducibility
model_yaml_path = output_dir / "model_config.yml"
save_model_to_yaml(gauss_model, model_yaml_path)
print(f"Model configuration saved to: {model_yaml_path}")

# Verify we can reload it
reloaded_model = create_model_from_yaml(model_yaml_path)
print(f"✅ Model successfully reloaded: {reloaded_model}")

print(f"Parameters match: {(reloaded_model.mu0 == gauss_model.mu0).all() and (reloaded_model.sigma0 == gauss_model.sigma0).all() and (reloaded_model.sigma == gauss_model.sigma).all()}")


--- Step 1.5: Save Model Configuration to YAML ---
Model configuration saved to: ../results/model_config.yml
✅ Model successfully reloaded: GaussGaussMultiDimModel(dim=2, n_obs=10, marginal_of_interest=1)
Parameters match: True


In [8]:
# --- Step 2: Save Simulator Configuration ---
print("\n--- Step 2: Save Simulator Configuration ---")
output_dir.mkdir(exist_ok=True, parents=True)
simulator.save(output_dir / "gauss_2D_marginal1_simulator.yml")


--- Step 2: Save Simulator Configuration ---
✅ Simulator saved with hash: ce73450df121
   - Configuration: ../results/gauss_2D_marginal1_simulator.yml
   - Observed Data: ../results/observed_data_eps_inf_ce73450df121.npy


In [9]:
# --- Step 2.5: Demonstrate YAML Model Configuration ---
print("\n--- Step 2.5: Demonstrate YAML Model Configuration ---")

# Show the complete model configuration
print("Current model configuration:")
model_config = gauss_model.get_model_args()

print("Model configuration:")
for key_arg, value in model_config.items():
    print(f"  {key_arg}: {value}")


--- Step 2.5: Demonstrate YAML Model Configuration ---
Current model configuration:
Model configuration:
  model_type: GaussGaussMultiDimModel
  model_class: GaussGaussMultiDimModel
  model_args: {'mu0': [0.0, 0.0], 'sigma0': [[4.0, 0.0], [0.0, 4.0]], 'sigma': [[0.25, 0.0], [0.0, 0.25]], 'dim': 2, 'n_obs': 10, 'marginal_of_interest': 1}
  metadata: {'parameter_space_dim': 2, 'analytical_posterior': True, 'module': 'abcnre.simulation.models.gauss_gauss'}


In [10]:
# --- Step 3: Create Network and Training Configuration ---

print("\n--- Step 3: Create Network Configuration ---")
nre_config = get_nre_config('conditioned_deepset_reduce_on_plateau')  # Utiliser une configuration existante
nre_config.training.num_epochs = 50
nre_config.training.n_samples_per_epoch = 10240
nre_config.training.num_thetas_to_store = 10000

nre_config.save(network_config_path)
print(f"Training configuration saved to: {network_config_path}")
print(f"Using configuration: {nre_config.experiment_name}")
print(f"Network type: {nre_config.network.network_type}")
print(f"LR Schedule: {nre_config.training.lr_scheduler.schedule_name}")


--- Step 3: Create Network Configuration ---
Training configuration saved to: ../results/config_mlp_reduce_on_plateau.yml
Using configuration: conditioned_deepset_reduce_on_plateau
Network type: ConditionedDeepSetNetwork
LR Schedule: reduce_on_plateau


In [ ]:
# --- Step 4: Training NRE for 1D Posterior p(φ|x) ---
print("\n--- Step 4: Training NRE for 1D Posterior p(φ|x) ---")
print(f"Training objective: Learn r(φ, s(x)) to estimate p(φ|x)")
print(f"φ = θ[{gauss_model.marginal_of_interest}] ∈ ℝ (1D parameter of interest)")
print(f"s(x) = summary statistics from 2D observations")

# Force restart imports to get the latest code
import sys
if 'abcnre.inference.networks.conditioned_deepset' in sys.modules:
    del sys.modules['abcnre.inference.networks.conditioned_deepset']
if 'abcnre.inference.networks.base' in sys.modules:
    del sys.modules['abcnre.inference.networks.base']

# IMPORTANT: Reload modules to get the fixed implementations
%reload_ext autoreload
from importlib import reload
import abcnre.simulation.models.gauss_gauss
reload(abcnre.simulation.models.gauss_gauss)

# Import fresh network modules
from abcnre.inference.networks.base import create_network_from_config

print("🔧 Reloaded model module with corrected discrepancy function")
print("🔧 Imported fresh network modules")

# Use the current simulator that has summary_stats_fn defined
print(f"Using current simulator with summary statistics")
print(f"Summary stats function available: {hasattr(simulator, 'summary_stats_fn')}")

# Test the discrepancy function with summary statistics
if hasattr(simulator, 'observed_summary_stats'):
    test_summary1 = simulator.observed_summary_stats
    key_test, subkey = jax.random.split(key)
    test_theta, test_x = simulator.model.sample_theta_x(subkey)
    test_summary2 = simulator.summary_stats_fn(test_x)

    distance = simulator.model.discrepancy_fn(test_summary1, test_summary2)
    print(f"🧪 Test discrepancy calculation: {distance:.4f}")
    print(f"   Summary 1 shape: {test_summary1.shape}")
    print(f"   Summary 2 shape: {test_summary2.shape}")
else:
    print("No observed summary statistics available")

# Create network and estimator
network = create_network_from_config(nre_config.network.to_dict())
estimator = NeuralRatioEstimator(
    network=network,
    training_config=nre_config.training,
    random_seed=nre_config.random_seed
)

print(f"Starting training for {nre_config.training.num_epochs} epochs...")
print(f"Each epoch: {nre_config.training.n_samples_per_epoch} samples")
print(f"Result: Trained NRE that can evaluate r(φ, s(x)) for any φ ∈ ℝ")

# Use current simulator for training (not loaded one)
key, train_key = jax.random.split(key)
training_history = estimator.train(
                                    key=train_key,
                                    simulator=simulator,  # Use current simulator
                                    output_dir=output_dir,
                                    num_epochs=nre_config.training.num_epochs,
                                    n_samples_per_epoch=nre_config.training.n_samples_per_epoch,
                                    batch_size=nre_config.training.batch_size
                                )

print("Training complete!")
print("NRE now estimates: p(φ|x) ∝ p(φ) × r(φ, s(x))")
print("Ready to evaluate 1D posterior for any φ value!")


--- Step 4: Training NRE for 1D Posterior p(φ|x) ---
Training objective: Learn r(φ, s(x)) to estimate p(φ|x)
φ = θ[1] ∈ ℝ (1D parameter of interest)
s(x) = summary statistics from 2D observations
🔧 Reloaded model module with corrected discrepancy function
🔧 Imported fresh network modules
Using current simulator with summary statistics
Summary stats function available: True
SHAPE X_FORMATTED: (1, 10, 2)
None
SHAPE X_FORMATTED: (1, 10, 2)
None
🧪 Test discrepancy calculation: 0.1548
   Summary 1 shape: (1, 1)
   Summary 2 shape: (1, 1)
Starting training for 50 epochs...
Each epoch: 10240 samples
Result: Trained NRE that can evaluate r(φ, s(x)) for any φ ∈ ℝ
🧪 Test discrepancy calculation: 0.1548
   Summary 1 shape: (1, 1)
   Summary 2 shape: (1, 1)
Starting training for 50 epochs...
Each epoch: 10240 samples
Result: Trained NRE that can evaluate r(φ, s(x)) for any φ ∈ ℝ
SHAPE X_FORMATTED: (1, 10, 2)
None
SHAPE X_FORMATTED: (1, 10, 2)
None
SHAPE X_FORMATTED: (1, 10, 2)
None
SHAPE X_FORMAT

In [12]:
# --- Step 5: Saving All Classifier Artifacts ---
print("\n--- Step 5: Saving All Classifier Artifacts ---")

final_config_path = save_classifier(
    estimator=estimator,
    simulator=simulator,
    output_dir=output_dir,
    filename_base="gauss_2D_marginal1"
)


--- Step 5: Saving All Classifier Artifacts ---
✅ Simulator saved with hash: ce73450df121
   - Configuration: ../results/gauss_2D_marginal1_simulator_eps_inf_1e62eb531277.yml
   - Observed Data: ../results/observed_data_eps_inf_ce73450df121.npy
✅ Simulator saved to: ../results/gauss_2D_marginal1_simulator_eps_inf_1e62eb531277.yml
⚠️  Note: accumulated_phi_samples not available (feature disabled for simplicity)
Flattened raw data shape: (1, 20)
Summary statistics available: True
Using pre-computed phi samples from sampler
Phi samples shape: (1, 1)
Summary statistics shape: (1, 1)
Final flattened features shape: (2, 22)
Raw data features span indices: [0, 1, 2, 3, 4]...[15, 16, 17, 18, 19]
Total raw data features: 20
Structured metadata created:
  original_data_shape: (10, 2)
  n_obs: 10
  data_dim: 2
  phi_dim: 1
  summary_dim: 1
  has_summary_stats: True
✅ Classifier saved to: ../results/gauss_2D_marginal1_classifier_eps_inf_03405013d665.yml
   - Hash: 03405013d665 (includes network +

In [13]:
# --- Step 6: Verification and Summary ---
print("\n--- Step 6: Training Complete - Summary ---")
print("🎯 OBJECTIVE ACHIEVED: Trained NRE for 1D posterior p(φ|x)")
print(f"✅ Parameter of interest: φ = θ[{gauss_model.marginal_of_interest}] ∈ ℝ")
print(f"✅ Input dimension: {simulator.observed_data.shape} (2D observations)")
print(f"✅ Network type: {type(network).__name__}")

print(f"\n📁 SAVED ARTIFACTS:")
print(f"   🔧 Final classifier config: {final_config_path}")
print(f"   📊 Simulator config: {simulator_path}")
print(f"   🎯 Network config: {network_config_path}")

print(f"\n📈 TRAINING RESULTS:")
# Check what's actually in training_history
print(f"   Training history keys: {list(training_history.keys())}")

# Use estimator's training_history instead
if hasattr(estimator, 'training_history') and estimator.training_history:
    history = estimator.training_history
    final_val_acc = history['val_accuracy'][-1] if 'val_accuracy' in history else "N/A"
    final_train_acc = history['train_accuracy'][-1] if 'train_accuracy' in history else "N/A"
    epochs_trained = len(history['train_loss']) if 'train_loss' in history else "N/A"
    
    print(f"   Epochs trained: {epochs_trained}")
    print(f"   Final validation accuracy: {final_val_acc}")
    print(f"   Final training accuracy: {final_train_acc}")
else:
    print(f"   Training completed successfully (detailed history not available)")

print(f"\n🚀 READY FOR INFERENCE:")
print(f"   Use the saved classifier config to load the trained model")
print(f"   Evaluate p(φ|x) for any φ value using the trained NRE")
print(f"   Compare against analytical posterior for validation")

# Check file existence
print(f"\n📋 FILE CHECK:")
files_to_check = [final_config_path, simulator_path, network_config_path]
for file_path in files_to_check:
    exists = file_path.exists()
    status = "✅" if exists else "❌"
    print(f"   {status} {file_path.name}: {exists}")

print(f"\n🎯 NEXT STEPS:")
print(f"   1. Open gauss_gauss_2D-load.ipynb")
print(f"   2. Load the trained classifier: {final_config_path.name}")
print(f"   3. Analyze the learned 1D posterior p(φ = θ[{gauss_model.marginal_of_interest}]|x)")
print(f"   4. Compare with analytical ground truth")


--- Step 6: Training Complete - Summary ---
🎯 OBJECTIVE ACHIEVED: Trained NRE for 1D posterior p(φ|x)
✅ Parameter of interest: φ = θ[1] ∈ ℝ
✅ Input dimension: (10, 2) (2D observations)
✅ Network type: ConditionedDeepSetNetwork

📁 SAVED ARTIFACTS:
   🔧 Final classifier config: ../results/gauss_2D_marginal1_classifier_eps_inf_03405013d665.yml
   📊 Simulator config: ../results/gauss_2D_marginal1_simulator.yml
   🎯 Network config: ../results/config_mlp_reduce_on_plateau.yml

📈 TRAINING RESULTS:
   Training history keys: ['history', 'final_train_loss', 'final_val_loss', 'final_train_accuracy', 'final_val_accuracy', 'epochs_trained', 'total_simulation_count', 'total_samples_processed']
   Epochs trained: 50
   Final validation accuracy: 0.84619140625
   Final training accuracy: 0.8359375

🚀 READY FOR INFERENCE:
   Use the saved classifier config to load the trained model
   Evaluate p(φ|x) for any φ value using the trained NRE
   Compare against analytical posterior for validation

📋 FILE C